In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 数据预处理
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

# 读取数据
root = 'image'
train_dataset = datasets.ImageFolder(root + '/train', transform)
test_dataset = datasets.ImageFolder(root + '/test', transform)

# 导入数据
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
classes = train_dataset.classes
classes_index = train_dataset.class_to_idx

# 创建两个模型，一个带有预训练参数，一个不带
model_pretrained = models.vgg16(pretrained=True)
model_random = models.vgg16(pretrained=False)

# 冻结预训练模型的特征提取层
for param in model_pretrained.features.parameters():
    param.requires_grad = False

# 替换新的全连接层
num_features = model_pretrained.classifier[0].in_features
model_pretrained.classifier = nn.Sequential(
    nn.Linear(num_features, 100),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(100, 2)
)

model_random.classifier = nn.Sequential(
    nn.Linear(num_features, 100),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(100, 2)
)

# 定义损失函数和优化器
entropy_loss = nn.CrossEntropyLoss()

optimizer_pretrained = optim.Adam(model_pretrained.classifier.parameters(), lr=0.0003)
optimizer_random = optim.Adam(model_random.classifier.parameters(), lr=0.0003)

# 训练和测试函数
def train(model, optimizer, train_loader):
    model.train()
    total_loss = 0
    correct = 0
    for inputs, labels in train_loader:
        out = model(inputs)
        loss = entropy_loss(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(out, 1)
        correct += (predicted == labels).sum().item()
    return correct / len(train_dataset), total_loss / len(train_loader)


def test(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            out = model(inputs)
            _, predicted = torch.max(out, 1)
            correct += (predicted == labels).sum().item()
    return correct / len(test_dataset)

# 训练并记录准确性
num_epochs = 10
train_acc_pretrained = []
test_acc_pretrained = []
train_acc_random = []
test_acc_random = []

for epoch in range(num_epochs):
    print('Epoch:', epoch)
    
    # 训练预训练模型
    acc_train_pre, _ = train(model_pretrained, optimizer_pretrained, train_loader)
    acc_test_pre = test(model_pretrained, test_loader)
    train_acc_pretrained.append(acc_train_pre)
    test_acc_pretrained.append(acc_test_pre)
    
    # 训练非预训练模型
    acc_train_rand, _ = train(model_random, optimizer_random, train_loader)
    acc_test_rand = test(model_random, test_loader)
    train_acc_random.append(acc_train_rand)
    test_acc_random.append(acc_test_rand)
    
    print(f'Pretrained - Train Acc: {acc_train_pre:.4f}, Test Acc: {acc_test_pre:.4f}')
    print(f'Random Init - Train Acc: {acc_train_rand:.4f}, Test Acc: {acc_test_rand:.4f}')

# 绘制结果对比图
plt.figure(figsize=(12, 6))
plt.plot(range(num_epochs), train_acc_pretrained, label='Pretrained - Train Acc')
plt.plot(range(num_epochs), test_acc_pretrained, label='Pretrained - Test Acc')
plt.plot(range(num_epochs), train_acc_random, label='Random Init - Train Acc')
plt.plot(range(num_epochs), test_acc_random, label='Random Init - Test Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Comparison of Training and Testing Accuracies')
plt.legend()
plt.show()

# 保存模型
torch.save(model_pretrained.state_dict(), 'cat_dog_pretrained.pth')
torch.save(model_random.state_dict(), 'cat_dog_random.pth')